In [3]:

# scraper_newsapi.py (Enhanced Version)

import os
import requests
import csv
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

API_KEY = os.getenv("NEWSAPI_KEY")
BASE_URL = "https://newsapi.org/v2"

def fetch_news_by_query(query):
    """Fetches news based on a search query using the /everything endpoint."""
    endpoint = "/everything"
    
    # Define headers for authentication (best practice)
    headers = {
        'X-Api-Key': "68f0b4a61fe644c7b90bcc655c84f826"
    }
    
    # Parameters for the request
    # See NewsAPI docs: https://newsapi.org/docs/endpoints/everything
    params = {
        'q': query,
        'language': 'en',
        'sortBy': 'publishedAt', # or 'relevancy', 'popularity'
        'pageSize': 20,
    }

    try:
        response = requests.get(BASE_URL + endpoint, headers=headers, params=params)
        response.raise_for_status()
        
        data = response.json()
        if data.get('status') != 'ok':
            print(f"API Error: {data.get('message')}")
            return []
            
        return data.get('articles', [])
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for query '{query}': {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

def save_articles_to_csv(articles, query):
    """Saves a list of NewsAPI articles to a CSV file."""
    if not articles:
        print("No articles to save.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"newsapi_{query.replace(' ', '_')}_{timestamp}.csv"

    headers = ['title', 'author', 'sourceName', 'description', 'url', 'publishedAt']

    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)
            
            for article in articles:
                writer.writerow([
                    article.get('title'),
                    article.get('author'),
                    article.get('source', {}).get('name'),
                    article.get('description'),
                    article.get('url'),
                    article.get('publishedAt'),
                ])
        print(f"Successfully saved {len(articles)} articles to {filename}")
    except IOError as e:
        print(f"Error writing to CSV file: {e}")

# Main execution block
if __name__ == "__main__":
    search_query = input("Enter a search query for news: ").strip()
    
    print(f"Searching for news about: '{search_query}' using NewsAPI.org")
    
    # 1. Fetch the news
    found_articles = fetch_news_by_query(search_query)
    
    # 2. Save the news to a CSV file
    if found_articles:
        save_articles_to_csv(found_articles, search_query)
    else:
        print("Could not find any articles for the given query.")

Searching for news about: 'Iran' using NewsAPI.org
Successfully saved 20 articles to newsapi_Iran_20250621_125414.csv
